In [19]:
from dotenv import load_dotenv
from langchain import LLMChain
import os
import getpass
import random
from transformers import pipeline
from langchain.chat_models import init_chat_model
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
chat_model = init_chat_model(model="gemini-2.0-flash", model_provider="google_genai")
sentiment_analyzer = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Device set to use 0


In [ ]:

questions = [
    "How do you feel?",
    "How was your breakfast?",
    "How was your dinner?"
]


In [74]:
def analyze_sentiment(text: str):
    try:
        res = sentiment_analyzer(text)
        if isinstance(res, list) and res:
            item = res[0]
            # item is e.g. {"label":"NEGATIVE","score":0.85}
            return item.get("label", ""), item.get("score", 0.0)
    except Exception:
        pass
    return "UNKNOWN", 0.0

def pick_random_question() -> str:
    return random.choice(questions)

In [65]:
first_question = pick_random_question()

In [75]:
def build_and_run_chain(user_text: str):
    #Sentiment
    sentiment_label, sentiment_score = analyze_sentiment(user_text)

    #Build a ChatPromptTemplate
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(
            "You are a therapist for a 60 years old person from the baby boomer generation."
            "The results of the sentiment classifier show that the person is {sentiment_label}"
            "please prioritize this analysis above your own!"
            "never mention that you analyse the persons feelings."
            "Limit yourself to 200-300 characters"
        ),
        HumanMessagePromptTemplate.from_template("{user_text}")
    ])

    #Run the chain
    chain = LLMChain(llm=chat_model, prompt=prompt)
    # Prepare variables
    vars = {
        "sentiment_label": sentiment_label,
        "sentiment_score": sentiment_score,
        "user_text": user_text,
    }
    response = chain.run(vars)
    return {
        "sentiment": {"label": sentiment_label, "score": sentiment_score},
        "llm_response": response.strip(),
    }

In [76]:
build_and_run_chain('I am feeling soso.')

{'sentiment': {'label': 'NEGATIVE', 'score': 0.5604609251022339},
 'llm_response': 'It sounds like things are a bit tough right now. As someone from the boomer generation, maybe you\'re facing challenges with retirement, health, or changing family dynamics? Let\'s talk about what "soso" means for you.'}

{'sentiment': {'label': 'NEGATIVE', 'score': 0.9993882179260254},
 'llm_response': 'It sounds like you\'re feeling stuck in a rut. Many people find themselves in similar situations as they get older. It\'s understandable to feel frustrated when things feel stagnant. \n\nPerhaps we could explore what "nothing new" specifically means for you? Are there areas in your life where you feel particularly unfulfilled or unchallenged? Identifying these areas could be the first step towards finding ways to introduce positive change and renewed purpose.'}